<a href="https://colab.research.google.com/github/aayush9753/LearningResources/blob/master/Level3_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install -qq transformers

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import WordCloud,STOPWORDS
stopwords = set(STOPWORDS)

from textblob import TextBlob

import re

from collections import Counter

import warnings
warnings.filterwarnings("ignore")

import os
import tensorflow as tf
from keras import Sequential
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torch.nn.functional as F

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('/content/gdrive/My Drive/Data/text_emotion_labelled.csv')

In [ ]:
data.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [ ]:
spotai = pd.read_csv('/content/gdrive/My Drive/Data/dataset.csv')

In [ ]:
spotai.head()

,Unnamed: 0,created_at,TEXT,LOCATION
0,0,2020-09-22 05:08:45,curve flattening kenya records 48 new virus ca...,IN
1,1,2020-09-22 05:08:34,victoria and melbourne covid trend map where c...,"Erbil, Iraq"
2,2,2020-09-22 05:08:33,nsw and sydney covid trend map where coronavir...,"Melbourne, Australia"
3,3,2020-09-22 05:06:02,it s bake off day who else will be tuning in a...,"Melbourne, Australia"
4,4,2020-09-22 05:05:34,the liberal party bots are out in force even t...,Fareham


In [ ]:
data = data.drop(['tweet_id','author'],axis=1)

In [ ]:
data.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [ ]:
spotai = spotai.rename(columns={'TEXT':'content','created_at':'time','LOCATION':'location'})
spotai.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
spotai.head()

,time,content,location
0,2020-09-22 05:08:45,curve flattening kenya records 48 new virus ca...,IN
1,2020-09-22 05:08:34,victoria and melbourne covid trend map where c...,"Erbil, Iraq"
2,2020-09-22 05:08:33,nsw and sydney covid trend map where coronavir...,"Melbourne, Australia"
3,2020-09-22 05:06:02,it s bake off day who else will be tuning in a...,"Melbourne, Australia"
4,2020-09-22 05:05:34,the liberal party bots are out in force even t...,Fareham


In [ ]:
def countries(df):
  names = []
  for i in df:
    k = i.split(', ')[-1]
    if k not in names:
      names.append(k)
  names = pd.DataFrame(names)
  return names

In [ ]:
# data preprocessing
for i in range(data.shape[0]) :
    data['content'][i] = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(#[A-Za-z0-9]+)", " ", data['content'][i]).split()).lower()
data['content'].head()

0    i know i was listenin to bad habit earlier and...
1    layin n bed with a headache ughhhh waitin on y...
2                       funeral ceremony gloomy friday
3                  wants to hang out with friends soon
4    we want to trade with someone who has houston ...
Name: content, dtype: object

In [ ]:
data.head()

,sentiment,content
0,empty,i know i was listenin to bad habit earlier and...
1,sadness,layin n bed with a headache ughhhh waitin on y...
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,wants to hang out with friends soon
4,neutral,we want to trade with someone who has houston ...


#Encoding the data

In [ ]:
emotion_id  = {"empty":0, "sadness":1,"enthusiasm":2,"neutral":3,"worry":4,
                        "surprise":5,"love":6,"fun":7,"hate":8,"happiness":9,"boredom":10,"relief":11,"anger":12}

In [ ]:
data["sentiment_id"] = data['sentiment'].map(emotion_id)

In [ ]:
data.head()

,sentiment,content,sentiment_id
0,empty,i know i was listenin to bad habit earlier and...,0
1,sadness,layin n bed with a headache ughhhh waitin on y...,1
2,sadness,funeral ceremony gloomy friday,1
3,enthusiasm,wants to hang out with friends soon,2
4,neutral,we want to trade with someone who has houston ...,3


In [ ]:
!pip install transformers

In [ ]:
from transformers import (
   AutoConfig,
   AutoTokenizer,
   TFAutoModelForSequenceClassification,
   AdamW,
   glue_convert_examples_to_features
)
import tensorflow as tf
import tensorflow_datasets as tfds
import json

In [ ]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data.sentiment_id)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
Y = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.content,Y, random_state=1995, test_size=0.2, shuffle=True)

In [ ]:
# using keras tokenizer here
token = tf.keras.preprocessing.text.Tokenizer(num_words=None)
max_len = 160
Epoch = 5
token.fit_on_texts(list(X_train) + list(X_test))
X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(token.texts_to_sequences(X_train), maxlen=max_len)
X_test_pad = tf.keras.preprocessing.sequence.pad_sequences(token.texts_to_sequences(X_test), maxlen=max_len)

In [ ]:
PRE_TRAINED_MODEL_NAME = 'digitalepidemiologylab/covid-twitter-bert'

# Initialise tokenizer
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
y_train[0].shape

(13,)

In [ ]:
tokenizer

In [ ]:
sample_txt = 'When was I last outside? I am stuck at home for 2 weeks.'

In [ ]:
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f' Sentence: {sample_txt}')
print(f'   Tokens: {tokens}')
print(f'Token IDs: {token_ids}')

 Sentence: When was I last outside? I am stuck at home for 2 weeks.
   Tokens: ['when', 'was', 'i', 'last', 'outside', '?', 'i', 'am', 'stuck', 'at', 'home', 'for', '2', 'weeks', '.']
Token IDs: [2043, 2001, 1045, 2197, 2648, 1029, 1045, 2572, 5881, 2012, 2188, 2005, 1016, 3134, 1012]


In [ ]:
encoding = tokenizer.encode_plus(
  sample_txt,
  max_length=32,
  add_special_tokens=True, # Add '[CLS]' and '[SEP]'
  return_token_type_ids=False,
  padding='max_length',
  return_attention_mask=True,
  return_tensors='pt',  # Return PyTorch tensors
)

encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
print(len(encoding['input_ids'][0]))
encoding['input_ids'][0]

32


tensor([ 101, 2043, 2001, 1045, 2197, 2648, 1029, 1045, 2572, 5881, 2012, 2188,
        2005, 1016, 3134, 1012,  102,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])

In [ ]:
df = data.copy().drop('sentiment',axis=1)

In [ ]:
df.head()

,content,sentiment_id
0,i know i was listenin to bad habit earlier and...,0
1,layin n bed with a headache ughhhh waitin on y...,1
2,funeral ceremony gloomy friday,1
3,wants to hang out with friends soon,2
4,we want to trade with someone who has houston ...,3


In [ ]:
tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])

['[CLS]',
 'when',
 'was',
 'i',
 'last',
 'outside',
 '?',
 'i',
 'am',
 'stuck',
 'at',
 'home',
 'for',
 '2',
 'weeks',
 '.',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

In [ ]:
MAX_LEN = 160

In [ ]:
class GPReviewDataset(Dataset):

  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.reviews)
  
  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      padding='max_length',
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'tweet': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED)
df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=RANDOM_SEED)

In [ ]:
df_train.shape, df_val.shape, df_test.shape

((36000, 2), (2000, 2), (2000, 2))

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = GPReviewDataset(
    reviews=df.content.to_numpy(),
    targets=df.sentiment_id.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )

In [ ]:
BATCH_SIZE = 16

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
data = next(iter(train_data_loader))
data.keys()

dict_keys(['tweet', 'input_ids', 'attention_mask', 'targets'])

In [ ]:
print(data['input_ids'].shape)
print(data['attention_mask'].shape)
print(data['targets'].shape)

torch.Size([16, 160])
torch.Size([16, 160])
torch.Size([16])


In [ ]:
bert_model = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [ ]:
last_hidden_state, pooled_output = bert_model(
  input_ids=encoding['input_ids'], 
  attention_mask=encoding['attention_mask']
)

In [ ]:
last_hidden_state.shape

torch.Size([1, 32, 1024])

In [ ]:
bert_model.config.hidden_size

1024

In [ ]:
pooled_output.shape

torch.Size([1, 1024])

In [ ]:
class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
num_classes = 13
model = SentimentClassifier(num_classes)
model = model.to(device)

In [ ]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)

print(input_ids.shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length

torch.Size([16, 160])
torch.Size([16, 160])


In [ ]:
F.softmax(model(input_ids, attention_mask), dim=1)

tensor([[0.0557, 0.1561, 0.0450, 0.2328, 0.0329, 0.0588, 0.0481, 0.0793, 0.0370,
         0.0660, 0.0465, 0.0664, 0.0753],
        [0.0676, 0.1859, 0.0347, 0.0988, 0.0259, 0.1011, 0.0375, 0.1036, 0.1313,
         0.0346, 0.0330, 0.0800, 0.0660],
        [0.0397, 0.0586, 0.0530, 0.1607, 0.0391, 0.1296, 0.0386, 0.0697, 0.0347,
         0.1395, 0.0493, 0.0804, 0.1071],
        [0.0409, 0.0279, 0.1633, 0.1533, 0.0550, 0.1084, 0.0915, 0.0791, 0.0300,
         0.0566, 0.0514, 0.0398, 0.1028],
        [0.0897, 0.1322, 0.0388, 0.2034, 0.0573, 0.0365, 0.0420, 0.0779, 0.0397,
         0.0628, 0.0799, 0.0874, 0.0524],
        [0.0827, 0.1918, 0.0391, 0.0841, 0.0228, 0.1232, 0.0389, 0.0854, 0.0448,
         0.1023, 0.0421, 0.1067, 0.0360],
        [0.0886, 0.1189, 0.0365, 0.1214, 0.0461, 0.1014, 0.0483, 0.0640, 0.0860,
         0.0594, 0.0387, 0.1263, 0.0644],
        [0.0288, 0.0656, 0.0703, 0.1138, 0.0409, 0.1465, 0.0519, 0.1162, 0.0344,
         0.0850, 0.1265, 0.0393, 0.0807],
        [0.0761,

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
EPOCHS = 10

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  scheduler, 
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0
  
  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)

    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
# %%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(df_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss = eval_model(
    model,
    val_data_loader,
    loss_fn, 
    device, 
    len(df_val)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

Epoch 1/10
----------


RuntimeError: ignored